In [15]:
using HTTP
using JSON
using JSON3
using Dates
using NCDatasets
using OrderedCollections
using PyPlot
const plt = PyPlot
using PyCall
mpl = pyimport("matplotlib")
mpl.style.use("./fairease.mplstyle")

The API URL kept in `.bashrc`...

In [2]:
beaconURL = ENV["beaconURL"]

"https://beacon-argo.maris.nl"

## Argo parameters and related units

In [3]:
parameters = ["sea_water_temperature",  "sea_water_salinity", 
        "mass_concentration_of_chlorophyll_a_in_sea_water", "moles_of_nitrate_per_unit_mass_in_sea_water"]
    
units = ["degree_Celsius", "psu", "mg/m3", "micromole/kg"]

4-element Vector{String}:
 "degree_Celsius"
 "psu"
 "mg/m3"
 "micromole/kg"

## Input fields

In [4]:
parameter = "sea_water_temperature" #Choose from Argo parameters and related units
unit = "degree_Celsius" #Choose from Argo parameters and related units --> make sure that it corresponds with the parameter
mintemporal = 63*365 #Days since 1950
maxtemporal = 72*365 #Days since 1950
minlon = 12.27
maxlon = 36.69
minlat = 28.26
maxlat = 47.43
regionname = "$(minlat)_$(minlon)_$(maxlat)_$(maxlon)" 
mindepth = 0. #Minimum water depth
maxdepth = 50. #Maximum water depth

50.0

## Query body based on input fields

In [6]:
myquery = prepare_query(parameter, unit, mintemporal, maxtemporal, mindepth, maxdepth, minlon, maxlon, minlat, maxlat)

"{\"query_parameters\":[{\"data_parameter\":\"sea_water_temperature\",\"unit\":\"degree_Celsius\",\"skip_fill_values\":true,\"alias\":\"sea_water_temperature\"},{\"data_parameter\":\"time\",\"unit\":\"days since 1950-01-01 00:00:00 UTC\",\"skip_fill_values\":false,\"alias\":\"TEMPORAL\"},{\"data_parame" ⋯ 326 bytes ⋯ ":\"TEMPORAL\"},\"min\":22995,\"max\":26280},{\"for_query_parameter\":{\"alias\":\"DEPTH\"},\"min\":0.0,\"max\":50.0},{\"for_query_parameter\":{\"alias\":\"LONGITUDE\"},\"min\":12.27,\"max\":36.69},{\"for_query_parameter\":{\"alias\":\"LATITUDE\"},\"min\":28.26,\"max\":47.43}],\"output\":{\"format\":\"netcdf\"}}"

### Perform request and write into netCDF file

In [16]:
filename = "Argo_$(parameter)_$(regionname)_$(Int(mintemporal/365+1950))-$(Int(maxtemporal/365+1950)-1)_[$(Int(mindepth))-$(Int(maxdepth))m].nc'"
@time open(filename, "w") do io
    HTTP.request("POST", "$(beaconURL)/api/query", 
    ["Content-Type" => "application/json"], myquery,
    response_stream=io);
end

  5.597722 seconds (110.33 k allocations: 133.818 MiB, 0.15% gc time, 0.16% compilation time)


HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Transfer-Encoding: chunked
Content-Type: application/x-netcdf
Content-Encoding: gzip
Last-Modified: Tue, 09 Apr 2024 13:26:50 GMT
Accept-Ranges: bytes
ETag: "2205cc:71e42ec:6615421a:101fc24"
Vary: accept-encoding, Origin, Access-Control-Request-Method, Access-Control-Request-Headers
content-disposition: attachment; filename="beacon_x1DHAN.nc"
X-Powered-By: ARR/3.0
Date: Tue, 09 Apr 2024 13:24:18 GMT

[Message Body was streamed]"""

### Read content from the netCDF file

read_netcdf (generic function with 1 method)

In [9]:
lon, lat, depth, dates, T =  read_netcdf("./test.nc");

## Plot results

In [10]:
pyimport("mpl_toolkits.basemap.Basemap")

LoadError: PyError (PyImport_ImportModule

The Python package mpl_toolkits.basemap.Basemap could not be imported by pyimport. Usually this means
that you did not install mpl_toolkits.basemap.Basemap in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the mpl_toolkits.basemap.Basemap module, you can
use `pyimport_conda("mpl_toolkits.basemap.Basemap", PKG)`, where PKG is the Anaconda
package that contains the module mpl_toolkits.basemap.Basemap, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run

In [11]:
using Conda
Conda.add("cartopy");
Conda.add("mpl_toolkits");

ccrs = pyimport("cartopy.crs")
cfeature = pyimport("cartopy.feature")
coast = cfeature.GSHHSFeature(scale="h")
dataproj = ccrs.PlateCarree();

[ Info: Running `conda install -y cartopy` in root environment


Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0


[ Info: Running `conda install -y mpl_toolkits` in root environment


Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - mpl_toolkits

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




LoadError: failed process: Process(setenv(`[4m/home/ctroupin/.julia/conda/3/bin/conda[24m [4minstall[24m [4m-y[24m [4mmpl_toolkits[24m`,["_CE_M=", "PATH=/home/ctroupin/conda_root/bin:/home/ctroupin/conda_root/condabin:/home/ctroupin/.gems/bin:/home/ctroupin/.local/bin:/home/ctroupin/bin:/usr/local/go/bin:/home/ctroupin/intel/oneapi/compiler/2022.1.0/linux/bin/intel64:/home/ctroupin/.juliaup/bin:/home/ctroupin/.local/bin:/home/ctroupin/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin:/home/ctroupin/.rvm/bin:/home/ctroupin/.rvm/bin", "QT_ACCESSIBILITY=1", "beaconURL=https://beacon-argo.maris.nl", "PAPERSIZE=a4", "LD_LIBRARY_PATH=/usr/local/lib:/opt/openssl/lib:/home/ctroupin/intel/oneapi/compiler/2022.1.0/linux/bin/intel64:/home/ctroupin/intel/oneapi/compiler/2022.1.0/linux/compiler/lib/intel64_lin:", "DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus", "GSETTINGS_SCHEMA_DIR_CONDA_BACKUP=", "XDG_SESSION_DESKTOP=u

In [12]:
fig = plt.figure(figsize = (12, 8))
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.set_extent([-20., 45., 30, 65])
scat = ax.scatter(lon, lat, s=3, c=T, cmap=plt.cm.RdYlBu_r)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                  linewidth=.5, color="gray", alpha=0.5, linestyle="--", zorder=3)
ax.add_feature(coast, lw=.5, color=".85", zorder=4)
gl.top_labels = false
gl.right_labels = false
ax.set_title("Argo $(parameter) $(regionname)\n$(Int(mintemporal/365+1950))-$(Int(maxtemporal/365+1950)-1)\n[$(mindepth)-$(maxdepth) m]")

cbar = plt.colorbar(scat, shrink=.65)
cbar.set_label("°C", rotation=0, ha="left")

plt.show()

LoadError: UndefVarError: `ccrs` not defined

In [13]:
Basemap = pyimport("mpl_toolkits.basemap.Basemap")


LoadError: PyError (PyImport_ImportModule

The Python package mpl_toolkits.basemap.Basemap could not be imported by pyimport. Usually this means
that you did not install mpl_toolkits.basemap.Basemap in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the mpl_toolkits.basemap.Basemap module, you can
use `pyimport_conda("mpl_toolkits.basemap.Basemap", PKG)`, where PKG is the Anaconda
package that contains the module mpl_toolkits.basemap.Basemap, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run

In [14]:
# Creating Basemap
m = Basemap(projection = 'cyl', llcrnrlon = -20, llcrnrlat = 30, urcrnrlon = 45, urcrnrlat = 65, resolution = 'i') 
m.drawlsmask(land_color = 'Linen', ocean_color = '#CCFFFF'); # can use HTML names or codes for colors
m.drawcoastlines()
m.drawcountries()

# Plot points
sc = m.scatter(df['LONGITUDE'], df['LATITUDE'], latlon = True, c = df[f'{parameter}'], cmap = plt.cm.jet)

plt.title(f'Argo {parameter} {regionname} {int(mintemporal/365+1950)}-{int(maxtemporal/365+1950)-1} [{mindepth}-{maxdepth}m]', fontsize = 25)

# Colorbar positioning
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size = "5%", pad = 0.05)
plt.colorbar(sc, cax = cax, label = f'{parameter} [{unit}]');

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///home/ctroupin/Projects/FAIR-EASE/DIVAnd-FAIR-EASE/In[14]#2:27\[90mIn[14]:2:27[0;0m]8;;\
# Creating Basemap
m = Basemap(projection = '[48;2;120;70;70mcyl[0;0m', llcrnrlon = -20, llcrnrlat = 30, urcrnrlon = 45, urcrnrlat = 65, resolution = 'i') 
[90m#                         └─┘ ── [0;0m[91mcharacter literal contains multiple characters[0;0m